In [ ]:
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
from matplotlib import pyplot as plt
torch.cuda.is_available()
import datetime
import torch.optim as optim
import torch.nn.functional as F
#Daniel Sullivan
#801095863

In [ ]:
from torchvision import datasets
data_path = 'drive/MyDrive/Colab Notebooks/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform = transforms.ToTensor())
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4195, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Problem 1

In [ ]:



train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1) # Part 2
    self.conv3 = nn.Conv2d(16, 8, kernel_size=3, padding=1)  
    self.fc1 = nn.Linear(4*4*8, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    out = F.max_pool2d(torch.tanh(self.conv2(out)), 2) # Part 2
    out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
    out = out.view(-1, 4*4*8)
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device='cuda')
      labels = labels.to(device='cuda')
      outputs = model(imgs)

      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

      loss_train += loss.item()

    if epoch == 1 or epoch % 10 == 0:
      print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [ ]:
model = Net().to(device = 'cuda')
optimizer = optim.SGD(model.parameters(), lr = 1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(300, optimizer, model, loss_fn, train_loader)

2022-03-23 14:29:42.579464 Epoch 1, Training Loss 2.3003357145792385
2022-03-23 14:31:41.120742 Epoch 10, Training Loss 1.546201981851817
2022-03-23 14:33:52.886411 Epoch 20, Training Loss 1.2697578250142314
2022-03-23 14:36:04.006029 Epoch 30, Training Loss 1.121033842834975
2022-03-23 14:38:14.051685 Epoch 40, Training Loss 1.0273683696146816
2022-03-23 14:40:24.591284 Epoch 50, Training Loss 0.9641874795374663
2022-03-23 14:42:34.812360 Epoch 60, Training Loss 0.9190347962977027
2022-03-23 14:44:44.140261 Epoch 70, Training Loss 0.884326749521753
2022-03-23 14:46:53.177298 Epoch 80, Training Loss 0.8577099882275857
2022-03-23 14:49:04.051202 Epoch 90, Training Loss 0.8340919273512443
2022-03-23 14:51:13.336031 Epoch 100, Training Loss 0.8163158984669029
2022-03-23 14:53:22.154276 Epoch 110, Training Loss 0.7999661336926853
2022-03-23 14:55:31.106005 Epoch 120, Training Loss 0.7862141566050936
2022-03-23 14:57:40.678758 Epoch 130, Training Loss 0.7723664066294575
2022-03-23 14:59:50.

In [ ]:
#train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

#val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0

    with torch.no_grad():
      for imgs, labels in loader:
        imgs = imgs.to(device='cuda')
        labels = labels.to(device='cuda')
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

    print("Accuracy {}: {:.2f}".format(name, correct / total))

#validate(model, train_loader, val_loader)

In [ ]:
#Problem 2

In [ ]:
class ResBlock(nn.Module):
  def __init__(self, n_chans):
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size = 3, padding = 1, bias = False)
    self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
    torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
    torch.nn.init.constant_(self.batch_norm.weight, 0.5)
    torch.nn.init.zeros_(self.batch_norm.bias)

  def forward(self, x):
    out = self.conv(x)
    out = self.batch_norm(out)
    out = torch.relu(out)
    return out + x

In [ ]:
class ResNet_10(nn.Module):
  def __init__(self, n_chans1 = 32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
    self.resblocks = nn.Sequential(*(n_blocks * [ResBlock(n_chans = n_chans1)]))
    self.fc1 = nn.Linear(8 *8 * n_chans1, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = self.resblocks(out)
    out = F.max_pool2d(out, 2)
    out = out.view(-1, 8 * 8 * self.n_chans1)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model2 = ResNet_10().to(device = 'cuda')
optimizer = optim.SGD(model2.parameters(), lr = 3e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(300, optimizer, model2, loss_fn, train_loader)

2022-03-23 15:35:11.071012 Epoch 1, Training Loss 1.805843689709978
2022-03-23 15:39:23.620004 Epoch 10, Training Loss 0.9612440843411418
2022-03-23 15:44:05.600064 Epoch 20, Training Loss 0.7624676719193568
2022-03-23 15:48:46.615787 Epoch 30, Training Loss 0.6327849361292847
2022-03-23 15:53:28.817180 Epoch 40, Training Loss 0.5267089729761834
2022-03-23 15:58:08.536401 Epoch 50, Training Loss 0.44487251598588035
2022-03-23 16:02:49.515674 Epoch 60, Training Loss 0.3750768093882924
2022-03-23 16:07:30.206053 Epoch 70, Training Loss 0.3109951575698755
2022-03-23 16:12:10.779778 Epoch 80, Training Loss 0.25670885155572915
2022-03-23 16:16:52.540677 Epoch 90, Training Loss 0.21688751167501025
2022-03-23 16:21:34.996040 Epoch 100, Training Loss 0.18954423415329297
2022-03-23 16:26:15.340669 Epoch 110, Training Loss 0.14127037594394992
2022-03-23 16:30:59.755591 Epoch 120, Training Loss 0.14085899077384445
2022-03-23 16:35:43.396529 Epoch 130, Training Loss 0.09663171721074511
2022-03-23 

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model2, train_loader, val_loader)

Accuracy train: 0.88
Accuracy val: 0.41


In [ ]:
class ResBlock_Drop(nn.Module):
  def __init__(self, n_chans):
    super(ResBlock_Drop, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size = 3, padding = 1, bias = False)
    self.conv_dropout = nn.Dropout2d(p = 0.3)

  def forward(self, x):
    out = self.conv(x)
    out = torch.relu(out)
    out = out + x
    out = self.conv_dropout(out)
    return out

In [ ]:
class ResNet_10_Drop(nn.Module):
  def __init__(self, n_chans1 = 32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
    self.resblocks = nn.Sequential(*(n_blocks * [ResBlock_Drop(n_chans = n_chans1)]))
    self.fc1 = nn.Linear(8 *8 * n_chans1, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = self.resblocks(out)
    out = F.max_pool2d(out, 2)
    out = out.view(-1, 8 * 8 * self.n_chans1)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model3 = ResNet_10_Drop().to(device = 'cuda')
optimizer = optim.SGD(model3.parameters(), lr = 3e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(300, optimizer, model3, loss_fn, train_loader)

2022-03-27 23:44:32.632116 Epoch 1, Training Loss 2.2939971868339404
2022-03-27 23:48:46.166564 Epoch 10, Training Loss 1.6863128201431021
2022-03-27 23:53:26.862375 Epoch 20, Training Loss 1.4977321261944978
2022-03-27 23:58:07.163457 Epoch 30, Training Loss 1.3949790778367415
2022-03-28 00:02:47.252744 Epoch 40, Training Loss 1.3307604270670421
2022-03-28 00:07:27.637759 Epoch 50, Training Loss 1.2760224964307703
2022-03-28 00:12:08.062812 Epoch 60, Training Loss 1.2474415838870856
2022-03-28 00:16:48.222933 Epoch 70, Training Loss 1.2208610536039943
2022-03-28 00:21:28.336857 Epoch 80, Training Loss 1.197258521223922
2022-03-28 00:26:08.308963 Epoch 90, Training Loss 1.1754911881121224
2022-03-28 00:30:48.122508 Epoch 100, Training Loss 1.1532476008548151
2022-03-28 00:35:27.960905 Epoch 110, Training Loss 1.136587511547996
2022-03-28 00:40:07.719954 Epoch 120, Training Loss 1.1210269153575458
2022-03-28 00:44:47.882089 Epoch 130, Training Loss 1.113270875438095
2022-03-28 00:49:28.

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model3, train_loader, val_loader)

Accuracy train: 0.65
Accuracy val: 0.32


In [ ]:
class ResBlock_Penalties(nn.Module):
  def __init__(self, n_chans):
    super(ResBlock_Penalties, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size = 3, padding = 1, bias = False)

  def forward(self, x):
    out = self.conv(x)
    out = torch.relu(out)
    return out + x

In [ ]:
def training_loop_penalties(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device='cuda')
      labels = labels.to(device='cuda')
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)

      l2_lambda = 0.001
      l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
      loss = loss + l2_lambda * l2_norm

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

      loss_train += loss.item()

    if epoch == 1 or epoch % 10 == 0:
      print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [ ]:
class ResNet_10_Penalties(nn.Module):
  def __init__(self, n_chans1 = 32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
    self.resblocks = nn.Sequential(*(n_blocks * [ResBlock_Penalties(n_chans = n_chans1)]))
    self.fc1 = nn.Linear(8 *8 * n_chans1, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = self.resblocks(out)
    out = F.max_pool2d(out, 2)
    out = out.view(-1, 8 * 8 * self.n_chans1)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model4 = ResNet_10_Penalties().to(device = 'cuda')
optimizer = optim.SGD(model4.parameters(), lr = 3e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop_penalties(300, optimizer, model4, loss_fn, train_loader)

2022-03-28 02:05:22.538298 Epoch 1, Training Loss 1.947956172706526
2022-03-28 02:09:54.502553 Epoch 10, Training Loss 1.037286742416489
2022-03-28 02:14:50.755368 Epoch 20, Training Loss 0.8038683915534592
2022-03-28 02:19:42.497645 Epoch 30, Training Loss 0.6535343272835398
2022-03-28 02:24:27.196270 Epoch 40, Training Loss 0.542217904535096
2022-03-28 02:29:11.824325 Epoch 50, Training Loss 0.4651967398155376
2022-03-28 02:33:56.132067 Epoch 60, Training Loss 0.40900400587741065
2022-03-28 02:38:41.090691 Epoch 70, Training Loss 0.3691396387222478
2022-03-28 02:43:26.815051 Epoch 80, Training Loss 0.34137875438117615
2022-03-28 02:48:12.122658 Epoch 90, Training Loss 0.32672781695414077
2022-03-28 02:52:55.255196 Epoch 100, Training Loss 0.3161836064723142
2022-03-28 02:57:37.539984 Epoch 110, Training Loss 0.3168284235631718
2022-03-28 03:02:18.045648 Epoch 120, Training Loss 0.2864150684088697
2022-03-28 03:06:58.954742 Epoch 130, Training Loss 0.2813523673188046
2022-03-28 03:11:

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

validate(model4, train_loader, val_loader)

Accuracy train: 0.92
Accuracy val: 0.43
